In [3]:
COVID_dir = '/Users/yiningliu/research/pooled-sampling/COVID-19-pooling' # set this to your directory
import sys
sys.path.insert(1, COVID_dir) 

In [4]:
import numpy as np
from numpy import genfromtxt
import scipy.io
from test import recover_pool_results, test_M, get_accuracy, get_num_errors
from optimal_sizes import minT
from membership_matrix import generate_const_col_weight, generate_doubly_regular_col
import matplotlib.pyplot as plt
import json
from util import simulate_x

# Input Parameters

In [5]:
num_samples = 384 
f = 1/384
fpr, fnr = 0, 0

# Default 
num_M = 25

In [7]:
print("For infection rate %.2f%%, please use at least %s pools." % (f * 100, minT(f, num_samples)))

For infection rate 0.26%, please use at least 11 pools.


In [8]:
T = 48

Generate simulated infection vectors. 

In [9]:
num_trials = 100
filename = "n%s-f%.4f-numTrials%s.csv" % (num_samples, f, num_trials)
simulate_x(num_samples, num_trials, f, filename) 

On average, 0.97 positives in each trail.


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
test_file = COVID_dir + '/tests/data/' + filename
for m in range(1, 11): 
    test_M(m, num_samples, T, f, fpr, fnr, num_M, num_trials, test_file)

Starting matrix 0
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 .

Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting matrix 15
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 

Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting matrix 0
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 ...
Starting trail 20 ...
Starting trail 30 ...
Starting trail 40 ...
Starting trail 50 ...
Starting trail 60 ...
Starting trail 70 ...
Starting trail 80 ...
Starting trail 90 ...
Starting trail 0 ...
Starting trail 10 .

KeyboardInterrupt: 

In [5]:
def plot_accuracy(k, error_type, generate_matrix):
    const_col_dir = COVID_dir + '/tests/results/' + generate_matrix + '/'
    
    x, y, average_acc = get_accuracy(COVID_dir, const_col_dir, n, k, T, num_trials, num_random_matrices, col_weights, error_type)

    plt.scatter(x, y)
    plt.xlabel('column weights')
    plt.ylabel('num_errors')
    title = 'Scatter plot for constant-column weight accuracy (f = %s/384) based on % s' % (k, error_type)
    plt.title(title)
    plt.show() 

    if error_type == 'num_errors':
        print("theoretical optimal column weight: %.3f" % optimal_column_weight(k/n, fnr, fpr, T, n))
        optimal_weight = col_weights[average_acc.index(np.max(average_acc))]
        print("empirical optimal column weight: %.3f" % optimal_weight)
        print("------------------------------------------------------------")